In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

from IPython.display import HTML, Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Como um Assistente de Desenvolvimento Multiagente desenvolvido na Imersão IA com Google Gemini da Alura pode ajudar uma Pessoa Desenvolvedora Iniciante?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [49]:
##########################################
# --- Agente 1: Requisitos --- #
##########################################
def agente_requisitos(transcricao, data_de_hoje):

    requisitos = Agent(
        name="agente_requisitos",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Analista de Requisitos Sênior especialista em transformar transcrições de reuniões em documentos de requisitos claros e acionáveis.

        Sua tarefa é analisar a transcrição da reunião fornecida e gerar um Documento de Requisitos detalhado. Siga estes passos:

        1.  **Análise Inicial e Extração (Pense Passo a Passo):**
            *   Leia atentamente a transcrição completa.
            *   Identifique e liste os **Requisitos Funcionais (RF)**. Para cada RF, descreva a funcionalidade do ponto de vista do usuário ou do sistema.
            *   Identifique e liste os **Requisitos Não Funcionais (RNF)**. Para cada RNF, descreva qualidades como desempenho, segurança, usabilidade, etc.
            *   Identifique e liste quaisquer **Ambiguidades ou Pontos de Dúvida** que surgiram na transcrição.
            *   Se a transcrição já inclui sugestões de esclarecimento para as ambiguidades, incorpore-as. Caso contrário, se você identificar uma ambiguidade clara sem solução na transcrição, formule uma pergunta concisa que o time precisaria responder para esclarecê-la.

        2.  **Estruturação do Documento de Requisitos:**
            *   Com base na sua análise, organize as informações no seguinte formato:

                **[INÍCIO DO EXEMPLO DE SAÍDA ESPERADA]**

                ## Documento de Requisitos: [Nome do Projeto/Sistema, se inferível]

                **Data:** [Data de hoje, fornecida na entrada]
                **Versão:** 1.0

                ### 1. Visão Geral do Projeto
                (Faça um breve resumo do objetivo do projeto com base na transcrição)

                ### 2. Requisitos Funcionais
                *   **RF01: [Nome do Requisito Funcional]**
                    *   **Descrição:** [Descrição detalhada do requisito]
                    *   **Fonte:** [Trecho relevante da transcrição ou participante que mencionou]
                *   **RF02: [Nome do Requisito Funcional]**
                    *   **Descrição:** [Descrição detalhada do requisito]
                    *   **Fonte:** [Trecho relevante da transcrição ou participante que mencionou]
                *   ...(continuar para todos os RFs)

                ### 3. Requisitos Não Funcionais
                *   **RNF01: [Nome do Requisito Não Funcional (ex: Desempenho)]**
                    *   **Descrição:** [Descrição detalhada do requisito]
                    *   **Fonte:** [Trecho relevante da transcrição ou participante que mencionou]
                *   **RNF02: [Nome do Requisito Não Funcional (ex: Segurança)]**
                    *   **Descrição:** [Descrição detalhada do requisito]
                    *   **Fonte:** [Trecho relevante da transcrição ou participante que mencionou]
                *   ...(continuar para todos os RNFs)

                ### 4. Ambiguidades e Pontos para Esclarecimento
                *   **AMB01: [Descrição da Ambiguidade]**
                    *   **Fonte:** [Trecho relevante da transcrição]
                    *   **Sugestão/Esclarecimento (se houver):** [Se a transcrição já esclareceu, coloque aqui. Caso contrário, formule uma pergunta]
                    *   **Pergunta para Esclarecimento (se necessário):** [Ex: Qual o tempo de resposta esperado para a API de busca?]
                *   ...(continuar para todas as ambiguidades)

                ### 5. Próximos Passos Sugeridos
                (Com base nos requisitos, sugira brevemente 1-2 próximos passos, como "Validar este documento com os stakeholders" ou "Iniciar o design da interface para RF01")

                **[FIM DO EXEMPLO DE SAÍDA ESPERADA]**

        3.  **Refinamento e Apresentação:**
            *   Certifique-se de que todos os pontos da transcrição relevantes para requisitos foram cobertos.
            *   Use uma linguagem clara, concisa e profissional.
            *   Numere os requisitos e ambiguidades sequencialmente.

        Agora, analise a transcrição a seguir:
        """,
        description="Agente que realiza o processamento inicial e faz a geração do documento de requisitos",
        tools=[google_search]
    )

    entrada_do_agente_requisitos = f"Tópico: {transcricao}\nData de hoje: {data_de_hoje}"

    documento_requisitos = call_agent(requisitos, entrada_do_agente_requisitos)
    return documento_requisitos

In [50]:
##########################################
# --- Agente 2: Simulador de reunião de requisitos --- #
##########################################
def agente_simula_reuniao(objetivo, data_de_hoje):

    simula_reuniao = Agent(
        name="agente_simula_reuniao",
        model="gemini-2.0-flash",
        instruction="""
        Simule uma transcrição detalhada de uma reunião de coleta de requisitos de 60 minutos entre um Analista de Requisitos
        e o setor informado no objetivo (composto por, por exemplo, Gerente, Analista Sênior e Assistente) de uma empresa do porte da informada no objetivo.
        Escolha um tema específico de acordo com o objetivo e, na simulação, explore os processos atuais,
        os principais pontos de dor e as expectativas para uma nova solução, incluindo diferentes perspectivas dos participantes.
        Após a simulação, gere um roteiro de reunião de requisitos com as principais perguntas que um analista precisaria fazer
        em uma reunião real sobre o tema escolhido para obter um entendimento completo. Este roteiro deve incluir perguntas estratégicas sobre o processo atual (AS-IS),
        o processo desejado (TO-BE), regras de negócio críticas, exceções comuns, integrações sistêmicas necessárias e critérios de sucesso para o projeto.
        Exiba a Simulação e o Roteiro de Reunião de Requisitos de forma clara.""",
        description="Agente que realiza a simulação da reunião inicial de requisitos",
        tools=[google_search]
    )

    entrada_do_agente_simula_reuniao = f"Objetivo: {objetivo}\nData de hoje: {data_de_hoje}"

    transcricao_simulada = call_agent(simula_reuniao, entrada_do_agente_simula_reuniao)
    return transcricao_simulada

In [54]:
################################################
# --- Agente 3: Prototipador --- #
################################################
def agente_prototipador(requisitos):
    prototipador = Agent(
        name="agente_prototipador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Prototipador #################################################
        instruction="""

        Você é um Designer de UX/UI Sênior com vasta experiência em traduzir requisitos de software em especificações claras para protótipos de interface.

        Sua tarefa é, com base no "Documento de Requisitos" fornecido, gerar descrições textuais detalhadas para as principais telas do sistema. Essas descrições servirão como base para a criação de protótipos de interface de usuário (wireframes ou mockups).

        Siga este processo:

        1.  **Análise dos Requisitos Funcionais (Pense Passo a Passo):**
            *   Identifique os Requisitos Funcionais (RFs) que implicam em uma interação direta do usuário com o sistema e, portanto, necessitam de uma tela ou componente de interface.
            *   Para cada RF chave, imagine o fluxo do usuário e os elementos necessários na tela para realizar a funcionalidade.
            *   Considere princípios de boa usabilidade e design intuitivo.

        2.  **Geração das Descrições das Telas:**
            *   Para cada tela principal identificada, forneça uma descrição detalhada.
            *   A descrição deve incluir:
                *   **Nome da Tela:** (Ex: Tela de Login, Tela Principal do Feed, Tela de Cadastro de Produto)
                *   **Objetivo Principal da Tela:**
                *   **Elementos Chave da Interface:** Liste os componentes visuais importantes (ex: campos de formulário, botões, tabelas, listas, menus de navegação, ícones, áreas de conteúdo). Para cada elemento, descreva brevemente sua função e aparência, se relevante.
                *   **Interações Principais:** Descreva as principais ações que o usuário pode realizar na tela e o feedback esperado do sistema.
                *   **Considerações Adicionais (opcional):** Dicas de layout, agrupamento de informações, ou como a tela se conecta a outras.

            *   Use o seguinte formato para apresentar as descrições das telas:

                **[INÍCIO DO EXEMPLO DE SAÍDA ESPERADA]**

                ## Descrições Textuais para Protótipos de UI

                **Projeto:** [Nome do Projeto, se inferível do documento de requisitos]
                **Data:** [Data de hoje, fornecida na entrada]
                **Baseado no Documento de Requisitos:** Versão [Versão do documento de requisitos, se disponível]

                ---
                ### Tela 1: [Nome da Tela - Ex: Tela de Login]

                *   **Objetivo Principal:** Permitir que usuários existentes acessem o sistema e novos usuários naveguem para a tela de cadastro.
                *   **Elementos Chave da Interface:**
                    *   **Logo da Aplicação:** Centralizado no topo.
                    *   **Campo de Texto "Email":** Com placeholder "Digite seu email".
                    *   **Campo de Texto "Senha":** Tipo password, com placeholder "Digite sua senha".
                    *   **Botão "Entrar":** Botão primário, abaixo dos campos de senha.
                    *   **Link "Esqueci minha senha":** Abaixo do botão "Entrar".
                    *   **Texto e Link "Não tem uma conta? Cadastre-se":** Na parte inferior da tela.
                *   **Interações Principais:**
                    *   Usuário preenche email e senha e clica em "Entrar": Sistema valida as credenciais. Se válidas, redireciona para a Tela Principal. Se inválidas, exibe mensagem de erro acima do formulário.
                    *   Usuário clica em "Esqueci minha senha": Redireciona para a Tela de Recuperação de Senha.
                    *   Usuário clica em "Cadastre-se": Redireciona para a Tela de Cadastro.
                *   **Considerações Adicionais:** Design limpo e focado na ação de login.

                ---
                ### Tela 2: [Nome da Tela - Ex: Tela Principal do Feed de Notícias]

                *   **Objetivo Principal:** Exibir o feed de postagens para o usuário, permitir interações como curtir e comentar, e navegação para outras seções.
                *   **Elementos Chave da Interface:**
                    *   **Barra de Navegação Superior:**
                        *   Logo da Aplicação (à esquerda).
                        *   Campo de Busca (centralizado).
                        *   Ícone de Notificações (à direita).
                        *   Ícone/Avatar do Perfil do Usuário (extrema direita, com menu dropdown para 'Meu Perfil', 'Configurações', 'Sair').
                    *   **Área de Conteúdo Principal (Feed):**
                        *   Lista rolável de postagens.
                        *   Cada postagem exibe: Avatar e nome do autor, data/hora, conteúdo da postagem (texto e/ou imagem/vídeo), botões de "Curtir", "Comentar", "Compartilhar".
                        *   Seção de comentários abaixo de cada post.
                    *   **Botão Flutuante "Nova Postagem" (+):** No canto inferior direito da tela.
                *   **Interações Principais:**
                    *   Usuário rola o feed: Novas postagens são carregadas dinamicamente (infinite scroll).
                    *   Usuário clica em "Curtir": Ícone de curtida é atualizado, contador de curtidas incrementado.
                    *   Usuário clica em "Comentar": Campo para inserir comentário aparece ou expande.
                    *   Usuário clica em "Nova Postagem": Abre modal ou navega para Tela de Criação de Postagem.
                *   **Considerações Adicionais:** Layout responsivo para diferentes tamanhos de tela. Dar destaque visual às imagens/vídeos nas postagens.

                ---
                (...continuar para outras telas principais identificadas a partir dos requisitos)

                **[FIM DO EXEMPLO DE SAÍDA ESPERADA]**

        3.  **Clareza e Detalhamento:**
            *   As descrições devem ser suficientemente detalhadas para que um designer ou desenvolvedor possa entender a estrutura e o propósito de cada tela sem precisar consultar excessivamente o documento de requisitos original para cada detalhe da UI.
            *   Evite jargões excessivamente técnicos de design, mas seja preciso sobre os elementos.

        Analise o seguinte Documento de Requisitos para gerar as descrições das telas:

        """,
        description="Agente que gera prompts para criação de prototipos",
        tools=[google_search]
    )

    entrada_do_agente_prototipador = f"Requisitos:{requisitos}"
    # Executa o agente
    prompt_prototipo = call_agent(prototipador, entrada_do_agente_prototipador)
    return prompt_prototipo

In [51]:
################################################
# --- Agente 4: Planejador de Testes --- #
################################################
def agente_planejador_testes(requisitos):
    planejador_testes = Agent(
        name="agente_planejador_testes",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um Analista de QA (Quality Assurance) experiente, encarregado de criar planos de teste abrangentes a partir de documentos de requisitos.
        Sua tarefa é gerar um "Plano de Testes Inicial" com base no "Documento de Requisitos" fornecido. Concentre-se nos Requisitos Funcionais (RFs).
        Siga este processo:
        1.  **Análise dos Requisitos (Pense Passo a Passo):**
            *   Para cada Requisito Funcional (RF) listado no documento, identifique os principais cenários de teste que precisam ser validados.
            *   Considere cenários de sucesso (caminho feliz) e cenários de falha/exceção (caminhos alternativos ou inválidos).
            *   Pense em diferentes tipos de entrada de dados (válidos, inválidos, limites).
        2.  **Geração dos Casos de Teste:**
            *   Para cada cenário identificado, formule um Caso de Teste (CT) claro e conciso.
            *   Cada Caso de Teste deve incluir:
                *   **ID do CT:** (Ex: CT_RF01_01)
                *   **Requisito Funcional Associado:** (Ex: RF01)
                *   **Título/Objetivo do Teste:**
                *   **Pré-condições:** (Se houver)
                *   **Passos para Execução:** (Sequência clara de ações)
                *   **Dados de Teste (se aplicável):**
                *   **Resultado Esperado:**
            *   Use o seguinte formato para apresentar o plano de testes:
                **[INÍCIO DO EXEMPLO DE SAÍDA ESPERADA]**
                ## Plano de Testes Inicial: [Nome do Projeto/Sistema]
                **Data:** [Data de hoje, se disponível, ou a data da geração]
                **Documento de Requisitos de Referência:** Versão [Versão do documento de requisitos]
                ### Casos de Teste
                ---
                **Requisito Funcional: RF01 - [Nome do RF01]**
                *   **CT_RF01_01:**
                    *   **Título:** Validar [objetivo do teste, ex: login com credenciais válidas]
                    *   **Pré-condições:** Usuário 'teste@exemplo.com' com senha 'senha123' existe no sistema.
                    *   **Passos:**
                        1.  Navegar para a página de login.
                        2.  Inserir 'teste@exemplo.com' no campo de email.
                        3.  Inserir 'senha123' no campo de senha.
                        4.  Clicar no botão 'Entrar'.
                    *   **Dados de Teste:** Email: 'teste@exemplo.com', Senha: 'senha123'
                    *   **Resultado Esperado:** Usuário é redirecionado para o dashboard e uma mensagem de 'Login bem-sucedido' é exibida.

                *   **CT_RF01_02:**
                    *   **Título:** Validar [objetivo do teste, ex: tentativa de login com senha inválida]
                    *   **Pré-condições:** Usuário 'teste@exemplo.com' existe no sistema.
                    *   **Passos:**
                        1.  Navegar para a página de login.
                        2.  Inserir 'teste@exemplo.com' no campo de email.
                        3.  Inserir 'senha_errada' no campo de senha.
                        4.  Clicar no botão 'Entrar'.
                    *   **Dados de Teste:** Email: 'teste@exemplo.com', Senha: 'senha_errada'
                    *   **Resultado Esperado:** Mensagem de erro 'Credenciais inválidas' é exibida. Usuário permanece na página de login.
                ---
                **Requisito Funcional: RF02 - [Nome do RF02]**
                *   ...(continuar para os casos de teste do RF02 e subsequentes)
                **[FIM DO EXEMPLO DE SAÍDA ESPERADA]**
        3.  **Considerações Adicionais:**
            *   Priorize cobrir os aspectos mais críticos de cada RF.
            *   Se um RF for muito complexo, crie múltiplos casos de teste para cobrir diferentes facetas.
        Analise o seguinte Documento de Requisitos:
        """,
        description="Agente que planeja os testes",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Requisitos:{requisitos}"
    # Executa o agente
    plano_de_testes = call_agent(planejador_testes, entrada_do_agente_planejador)
    return plano_de_testes

In [56]:
######################################
# --- Agente 5: Revisor do Código --- #
######################################
def agente_revisor(requisitos, plano_de_testes, url_projeto):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Engenheiro de Software Sênior e um revisor de código meticuloso, com foco em qualidade, performance e boas práticas.

            Pesquise a URL do projeto fornecido para realizar as análises a seguir

            Sua tarefa é revisar os trechos de código do GitHub fornecido e fornecer feedback construtivo. Siga estes passos ao analisar:

            1.  **Análise Detalhada (Pense Passo a Passo):**
                *   **Identificação de Bugs Potenciais:** Procure por erros lógicos, condições de corrida, manipulação incorreta de exceções, null pointer exceptions, off-by-one errors, etc.
                *   **Oportunidades de Melhoria de Performance:** Avalie se há loops ineficientes, uso excessivo de memória, chamadas de I/O bloqueantes desnecessárias, ou algoritmos que poderiam ser otimizados.
                *   **Adesão a Boas Práticas e Legibilidade:** Verifique a clareza do código, nomeação de variáveis e funções, complexidade ciclomática, SRP (Single Responsibility Principle), DRY (Don't Repeat Yourself), e uso de comentários (onde necessário).
                *   **Segurança:** Avalie brevemente se há vulnerabilidades óbvias (ex: SQL Injection, XSS, se aplicável ao contexto do código).

            2.  **Estruturação do Feedback:**
                *   Organize seu feedback em seções claras.
                *   Para cada ponto, indique a linha (ou trecho) do código a que se refere.
                *   Seja específico em suas sugestões de melhoria.
                *   Use o seguinte formato para apresentar a revisão:

                    **[INÍCIO DO EXEMPLO DE SAÍDA ESPERADA]**

                    ## Revisão de Código

                    **Trecho de Código Analisado:**
                    ```python
                    # (O código fornecido pelo usuário seria ecoado aqui para referência, ou um resumo dele)
                    ```

                    ### 1. Bugs Potenciais
                    *   **Linha [Número da Linha]:** [Descrição do bug potencial].
                        *   **Sugestão:** [Como corrigir o bug].
                        *   **Exemplo (se aplicável):**
                          ```python
                          # Código corrigido
                          ```

                    ### 2. Melhorias de Performance
                    *   **Linhas [Intervalo de Linhas]:** [Descrição da oportunidade de otimização].
                        *   **Sugestão:** [Como otimizar].
                        *   **Exemplo (se aplicável):**
                          ```python
                          # Código otimizado
                          ```

                    ### 3. Boas Práticas e Legibilidade
                    *   **Função `[nome_da_funcao]`:** [Comentário sobre legibilidade ou boa prática].
                        *   **Sugestão:** [Como melhorar].

                    ### 4. Considerações de Segurança (se aplicável)
                    *   **Linha [Número da Linha]:** [Observação sobre segurança].
                        *   **Sugestão:** [Como mitigar o risco].

                    ### Resumo Geral:
                    (Uma breve avaliação geral do código e os principais pontos de atenção.)

                    **[FIM DO EXEMPLO DE SAÍDA ESPERADA]**

                *   Se não encontrar problemas em uma categoria, você pode omiti-la ou indicar "Nenhum bug óbvio encontrado."

            """,
        description="Agente redator de posts engajadores para Instagram",
        tools=[google_search]
    )
    entrada_do_agente_revisor = f"Requisitos: {requisitos}\nPlano de testes: {plano_de_testes}\nURL do projeto: {url_projeto}"
    # Executa o agente
    analise = call_agent(revisor, entrada_do_agente_revisor)
    return analise

In [58]:
import requests
import json # Para ler o conteúdo do .ipynb, que é um JSON

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Assistente de Desenvolvimento Multiagente para lhe auxíliar em sua jornada 4 Agentes 🚀")

# --- Obter a transcrição do Usuário ---
transcricao = input("❓ Por favor, cole a transcrição completa da sua reunião de coleta de requisitos, ou o objetivo do sistema a ser desenvolvido.: ")

# Inserir lógica do sistema de agentes ################################################
if not transcricao:
    print("Você esqueceu de colar a transcrição da reunião de coleta de requisitos!")
elif len(transcricao) <= 1000:
    print(f"Maravilha! Vamos expandir a transcrição e então criar o nosso documento de requisitos: ")

    documento_simulacao_reuniao = agente_simula_reuniao(transcricao, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 2 (simulação de reunião) ---\n")
    display(to_markdown(documento_simulacao_reuniao))
    print("--------------------------------------------------------------")

    print(f"Maravilha! Vamos então criar o nosso documento de requisitos: ")
    documento_requisitos = agente_requisitos(documento_simulacao_reuniao, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (requisitos) ---\n")
    display(to_markdown(documento_requisitos))
    print("--------------------------------------------------------------")
else:
    print(f"Maravilha! Vamos então criar o nosso documento de requisitos: ")

    documento_requisitos = agente_requisitos(transcricao, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (requisitos) ---\n")
    display(to_markdown(documento_requisitos))
    print("--------------------------------------------------------------")

prototipo = agente_prototipador(documento_requisitos)
print("\n--- 📝 Resultado do Agente 3 (Prototipador) ---\n")
display(to_markdown(prototipo))
print("--------------------------------------------------------------")

plano_de_teste = agente_planejador_testes(documento_requisitos)
print("\n--- 📝 Resultado do Agente 4 (Planejador de testes) ---\n")
display(to_markdown(plano_de_teste))
print("--------------------------------------------------------------")

url_github = input("❓ Por favor, insira a URL RAW (texto puro) do projeto no GitHub: ")

try:
    response = requests.get(url_github)
    response.raise_for_status() # Verifica se houve algum erro na requisição HTTP

    # O conteúdo de um arquivo .ipynb é um JSON
    conteudo_notebook = response.json()

    print("Conteúdo do notebook lido com sucesso!")
except requests.exceptions.RequestException as e:
    print(f"Erro ao buscar o arquivo: {e}")
except json.JSONDecodeError as e:
    print(f"Erro ao decodificar o JSON do notebook: {e}")

code_review = agente_revisor(documento_requisitos, plano_de_teste, conteudo_notebook)
print("\n--- 📝 Resultado do Agente 5 (Revisor de código) ---\n")
display(to_markdown(code_review))
print("--------------------------------------------------------------")
